<a href="https://colab.research.google.com/github/valid999/NLP-TensorFlow_Projects/blob/main/LSTM_Practical_information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

df = pd.read_csv('train.csv')

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
# Get the Independent features
X = df.drop('label' , axis = 1)

In [9]:
# Get the Dependent features
y = df['label']

In [10]:
X.shape

(20800, 4)

In [11]:
y.shape

(20800,)

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.14.0'

In [33]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [15]:
### Vocaboulary size
voc_size = 5000

In [16]:
## Onehot Represe4ntation
messages = X.copy()

In [17]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
messages.reset_index(inplace = True)

In [19]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
20795,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [20]:
messages.drop(['id'] , axis = 1)

,index,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [21]:
import nltk
import re
from nltk.corpus import stopwords

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', str(messages['title'][i]))
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [28]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find hom

In [29]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [35]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr[0]

[133, 4653, 3082, 4044, 546, 4347, 4372, 896, 1620, 124]

## Embedding Representation

In [36]:
sent_length = 20
embedding_docs = pad_sequences(onehot_repr , padding = 'pre' , maxlen = sent_length)
print(embedding_docs)

[[   0    0    0 ...  896 1620  124]
 [   0    0    0 ... 2539 4165 2552]
 [   0    0    0 ... 1707 3834  207]
 ...
 [   0    0    0 ... 1498 4390 2752]
 [   0    0    0 ... 3387 3364 2722]
 [   0    0    0 ...  788 3607 2810]]


In [37]:
embedding_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  133,
       4653, 3082, 4044,  546, 4347, 4372,  896, 1620,  124], dtype=int32)

In [38]:
## Creating model
embedding_vector_features = 40  ##  features representations
model = Sequential()
model.add(Embedding(voc_size , embedding_vector_features , input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1 , activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [40]:
len(embedding_docs) , y.shape

(20800, (20800,))

In [41]:
import numpy as np
X_final = np.array(embedding_docs)
y_final = np.array(y)

In [42]:
X_final.shape , y_final.shape

((20800, 20), (20800,))

In [43]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X_final , y_final , test_size = 0.33 , random_state = 42)

In [44]:
## Model Training
model.fit(X_train , y_train , validation_data = (X_test , y_test) , epochs = 10 , batch_size = 64)

Epoch 1/10
218/218 [==============================] - 24s 80ms/step - loss: 0.3234 - accuracy: 0.8523 - val_loss: 0.1839 - val_accuracy: 0.9258
Epoch 2/10
218/218 [==============================] - 4s 20ms/step - loss: 0.1316 - accuracy: 0.9467 - val_loss: 0.1877 - val_accuracy: 0.9305
Epoch 3/10
218/218 [==============================] - 2s 10ms/step - loss: 0.0864 - accuracy: 0.9691 - val_loss: 0.1977 - val_accuracy: 0.9280
Epoch 4/10
218/218 [==============================] - 3s 12ms/step - loss: 0.0581 - accuracy: 0.9799 - val_loss: 0.2308 - val_accuracy: 0.9202
Epoch 5/10
218/218 [==============================] - 2s 10ms/step - loss: 0.0352 - accuracy: 0.9890 - val_loss: 0.3287 - val_accuracy: 0.9123
Epoch 6/10
218/218 [==============================] - 2s 9ms/step - loss: 0.0226 - accuracy: 0.9917 - val_loss: 0.2955 - val_accuracy: 0.9261
Epoch 7/10
218/218 [==============================] - 2s 10ms/step - loss: 0.0147 - accuracy: 0.9955 - val_loss: 0.4123 - val_accuracy: 0.9167

### Adding Dropout

In [58]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [59]:
## Model Training
model.fit(X_train , y_train , validation_data = (X_test , y_test) , epochs = 10 , batch_size = 64)

Epoch 1/10
218/218 [==============================] - 13s 44ms/step - loss: 0.3291 - accuracy: 0.8424 - val_loss: 0.1868 - val_accuracy: 0.9237
Epoch 2/10
218/218 [==============================] - 4s 16ms/step - loss: 0.1390 - accuracy: 0.9438 - val_loss: 0.1815 - val_accuracy: 0.9276
Epoch 3/10
218/218 [==============================] - 2s 9ms/step - loss: 0.0984 - accuracy: 0.9633 - val_loss: 0.1882 - val_accuracy: 0.9251
Epoch 4/10
218/218 [==============================] - 2s 10ms/step - loss: 0.0702 - accuracy: 0.9757 - val_loss: 0.2243 - val_accuracy: 0.9293
Epoch 5/10
218/218 [==============================] - 2s 8ms/step - loss: 0.0489 - accuracy: 0.9828 - val_loss: 0.2432 - val_accuracy: 0.9193
Epoch 6/10
218/218 [==============================] - 3s 13ms/step - loss: 0.0352 - accuracy: 0.9884 - val_loss: 0.2946 - val_accuracy: 0.9272
Epoch 7/10
218/218 [==============================] - 2s 11ms/step - loss: 0.0269 - accuracy: 0.9907 - val_loss: 0.3150 - val_accuracy: 0.9216


## Performance of Metrics And Accuracy

In [60]:
y_pred=model.predict(X_test)

215/215 [==============================] - 1s 2ms/step


In [61]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [62]:
from sklearn.metrics import confusion_matrix

In [63]:
confusion_matrix(y_test,y_pred)

array([[3131,  318],
       [ 203, 3212]])

In [64]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9240967365967366

In [65]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92      3449
           1       0.91      0.94      0.92      3415

    accuracy                           0.92      6864
   macro avg       0.92      0.92      0.92      6864
weighted avg       0.92      0.92      0.92      6864

